In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# Load Data

In [2]:
def load_scenario_data(h5_file_path, scenario_num):
    key = f"scenario_{scenario_num}/data"
    with pd.HDFStore(h5_file_path, mode='r') as store:
        if key not in store:
            return None  # Scenario not found
        df = store[key]
        metadata = store.get_storer(key).attrs.metadata
    return {"dataset": df, "metadata": metadata}

In [3]:
store_files = [
    "../synthetic_data/RCT_0_5.h5",
    "../synthetic_data/RCT_0_05.h5",
    "../synthetic_data/e_X.h5",
    "../synthetic_data/e_X_U.h5",
    "../synthetic_data/e_X_no_overlap.h5",
    "../synthetic_data/e_X_info_censor.h5",
    "../synthetic_data/e_X_U_info_censor.h5",
    "../synthetic_data/e_X_no_overlap_info_censor.h5"
]

experiment_setups = {}

for path in store_files:
    base_name = os.path.splitext(os.path.basename(path))[0]  # e.g. RCT_0_5
    scenario_dict = {}
    for scenario in range(1, 11):
        try:
            result = load_scenario_data(path, scenario)
            if result is not None:
                scenario_dict[f"scenario_{scenario}"] = result
        except Exception as e:
            # Log or ignore as needed
            continue
    experiment_setups[base_name] = scenario_dict

In [4]:
experiment_setups['RCT_0_5']['scenario_1']['dataset'].head()

,id,observed_time,event,W,X1,X2,X3,X4,X5,U1,U2,T0,T1,T,C
0,0,0.054267,1,0,0.135488,0.887852,0.932606,0.445568,0.388236,0.151609,0.205535,0.054267,0.061394,0.054267,1.803019
1,1,0.732630,1,1,0.257596,0.657368,0.492617,0.964238,0.800984,0.597208,0.255785,0.228566,0.732630,0.732630,1.689546
2,2,0.162856,1,1,0.455205,0.801058,0.041718,0.769458,0.003171,0.370382,0.223214,0.176016,0.162856,0.162856,1.256329
3,3,0.050340,1,1,0.292809,0.610914,0.913027,0.300115,0.248599,0.038464,0.409829,0.381909,0.050340,0.050340,1.241777
4,4,0.524607,1,0,0.666392,0.987533,0.468270,0.123287,0.916031,0.342961,0.791330,0.524607,1.121968,0.524607,1.516613


In [5]:
experiment_repeat_setups = pd.read_csv("../synthetic_data/idx_split.csv").set_index("idx")
experiment_repeat_setups

,random_idx0,random_idx1,random_idx2,random_idx3,random_idx4,random_idx5,random_idx6,random_idx7,random_idx8,random_idx9
idx,,,,,,,,,,
0,47390,5618,14210,46970,4203,16369,24535,45204,45725,45885
1,38566,46218,39045,7253,22759,34401,28889,38471,45822,37471
2,32814,20226,40012,4854,27351,39165,25359,14516,25717,29860
3,41393,39492,27153,19041,33009,19822,21243,41228,955,23901
4,12564,17823,48976,18458,22756,28169,45851,36620,29824,12711
...,...,...,...,...,...,...,...,...,...,...
49995,15948,39245,30779,48178,45056,4892,528,7486,31042,38267
49996,11102,29624,40779,3136,45904,41903,45682,36621,33204,38070
49997,16338,8986,19293,35651,10172,17947,38843,18310,2765,12581


# EXPERIMENT CONSTANTS

In [6]:
num_repeats_to_include = 10  # max 10
num_training_data_points = 500 # max 45000
test_size = 5000
imputation_method = 'Pseudo_obs' # 'Pseudo_obs', 'Margin', 'IPCW-T'
meta_learner_type = 'x_learner'
load_imputed_values = True
imputed_times_path = f"../synthetic_data/imputed_times_lookup.pkl"

In [7]:
output_pickle_path = f"results/{meta_learner_type}_{imputation_method}_num_repeats_{num_repeats_to_include}_train_size_{num_training_data_points}.pkl"

# Run Experiments

In [8]:
import sys
import os

# Add the parent directory of "notebooks" to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from models_causal_impute_meta.meta_learners import TLearner, SLearner, XLearner
from models_causal_impute_meta.survival_eval_impute import SurvivalEvalImputer
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [9]:
def prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list, num_training_data_points=5000, test_size=5000):
    split_results = {}

    for rand_idx in random_idx_col_list:
        random_idx = experiment_repeat_setups[rand_idx].values
        test_ids = random_idx[-test_size:]
        train_ids = random_idx[:min(num_training_data_points, len(random_idx) - test_size)]

        X_cols = [c for c in dataset_df.columns if c.startswith("X") and c[1:].isdigit()]
        
        train_df = dataset_df[dataset_df['id'].isin(train_ids)]
        test_df = dataset_df[dataset_df['id'].isin(test_ids)]

        X_train = train_df[X_cols].to_numpy()
        W_train = train_df["W"].to_numpy()
        Y_train = train_df[["observed_time", "event"]].to_numpy()

        X_test = test_df[X_cols].to_numpy()
        W_test = test_df["W"].to_numpy()
        Y_test = test_df[["observed_time", "event"]].to_numpy()

        cate_test_true = (test_df["T1"] - test_df["T0"]).to_numpy()

        split_results[rand_idx] = (X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true)

    return split_results

In [10]:
random_idx_col_list = experiment_repeat_setups.columns.to_list()[:num_repeats_to_include]

print("Output results path:", output_pickle_path)

base_regressors = ['ridge', 'lasso', 'rf', 'gbr', 'xgb']

results_dict = {}

for setup_name, setup_dict in tqdm(experiment_setups.items(), desc="Experiment Setups"):

    results_dict[setup_name] = {}

    for scenario_key in tqdm(setup_dict, desc=f"{setup_name} Scenarios"):
        dataset_df = setup_dict[scenario_key]["dataset"]
        split_dict = prepare_data_split(dataset_df, experiment_repeat_setups, random_idx_col_list, num_training_data_points, test_size)

        # Initialize results dictionary for this setup and scenario
        results_dict[setup_name][scenario_key] = {}

        # For each base model, we will run the TLearner
        for base_model in tqdm(base_regressors, desc="Base Models", leave=False):
            # print(f"Running {base_model} for {setup_name} - {scenario_key}")
            
            # Store placeholder for later population
            results_dict[setup_name][scenario_key][base_model] = {}
            
            start_time = time.time()

            for rand_idx in random_idx_col_list:
                X_train, W_train, Y_train, X_test, W_test, Y_test, cate_test_true = split_dict[rand_idx]

                Y_train_imputed, Y_test_imputed = None, None
                if load_imputed_values:
                    # Load imputed values
                    with open(imputed_times_path, "rb") as f:
                        imputed_times = pickle.load(f)

                    # Get imputed values for the current random index
                    imputed_results = imputed_times.get(imputation_method, {}).get(setup_name, {}).get(scenario_key, {}).get(num_training_data_points, {}).get(rand_idx, {})
                    Y_train_imputed = imputed_results.get("Y_train_imputed", None)
                    Y_test_imputed = imputed_results.get("Y_test_imputed", None)
                
                # If imputed values are not loaded, use the original Y_train and Y_test
                if Y_train_imputed is None:
                    # impute the missing values
                    print(f"[Train] Imputing missing values for {setup_name} - {scenario_key} - {base_model} - {rand_idx}")
                    survival_imputer = SurvivalEvalImputer(imputation_method=imputation_method, verbose=False)
                    Y_train_imputed, Y_test_imputed = survival_imputer.fit_transform(Y_train, Y_test)
                
                if Y_test_imputed is None:
                    # impute the missing values
                    print(f"[Test] Imputing missing values for {setup_name} - {scenario_key} - {base_model} - {rand_idx}")
                    survival_imputer = SurvivalEvalImputer(imputation_method=imputation_method, verbose=False)
                    _, Y_test_imputed = survival_imputer.fit_transform(Y_train, Y_test, impute_train=False)
                
                if meta_learner_type == 't_learner':
                    learner = TLearner(base_model_name=base_model)
                elif meta_learner_type == 's_learner':
                    learner = SLearner(base_model_name=base_model)
                elif meta_learner_type == 'x_learner':
                    learner = XLearner(base_model_name=base_model)
                else:
                    raise ValueError(f"Unknown meta-learner type: {meta_learner_type}")
                
                learner.fit(X_train, W_train, Y_train_imputed)
                mse_test, cate_test_pred, ate_test_pred = learner.evaluate(X_test, cate_test_true, W_test)

                # Evaluate base survival models on test data
                base_model_eval = learner.evaluate_test(X_test, Y_test_imputed, W_test)

                # Save results
                results_dict[setup_name][scenario_key][base_model][rand_idx] = {
                    "cate_true": cate_test_true,
                    "cate_pred": cate_test_pred,
                    "ate_true": cate_test_true.mean(),
                    "ate_pred": ate_test_pred,
                    "cate_mse": mse_test,
                    "ate_bias": ate_test_pred - cate_test_true.mean(),
                    "base_model_eval": base_model_eval, # Store base model evaluation results
                }

            end_time = time.time()

            # Save results to the setup dictionary
            avg = results_dict[setup_name][scenario_key][base_model]
            base_model_eval_performance = {
                                            base_model_k: 
                                            {
                                                f"{stat}_{metric_j}": func([
                                                    avg[i]['base_model_eval'][base_model_k][metric_j] for i in random_idx_col_list
                                                    if i in avg
                                                ])
                                                for metric_j in metric_j_dict
                                                for stat, func in zip(['mean', 'std'], [np.nanmean, np.nanstd])
                                            }
                                            for base_model_k, metric_j_dict in avg[list(avg.keys())[0]]['base_model_eval'].items()
                                        }

            results_dict[setup_name][scenario_key][base_model]["average"] = {
                    "mean_cate_mse": np.mean([avg[i]["cate_mse"] for i in random_idx_col_list]),
                    "std_cate_mse": np.std([avg[i]["cate_mse"] for i in random_idx_col_list]),
                    "mean_ate_pred": np.mean([avg[i]["ate_pred"] for i in random_idx_col_list]),
                    "std_ate_pred": np.std([avg[i]["ate_pred"] for i in random_idx_col_list]),
                    "mean_ate_true": np.mean([avg[i]["ate_true"] for i in random_idx_col_list]),
                    "std_ate_true": np.std([avg[i]["ate_true"] for i in random_idx_col_list]),
                    "mean_ate_bias": np.mean([avg[i]["ate_bias"] for i in random_idx_col_list]),
                    "std_ate_bias": np.std([avg[i]["ate_bias"] for i in random_idx_col_list]),
                    "runtime": (end_time - start_time) / len(random_idx_col_list),
                    "base_model_eval": base_model_eval_performance
                }

            # Save progress to disk
            # with open(output_pickle_path, "wb") as f:
                # pickle.dump(results_dict, f)
            
            break
        break
    break
            


Output results path: results/x_learner_Pseudo_obs_num_repeats_10_train_size_500.pkl


Experiment Setups:   0%|          | 0/8 [00:00<?, ?it/s]

Experiment Setups:   0%|          | 0/8 [00:06<?, ?it/s]


In [11]:
results_dict

{'RCT_0_5': {'scenario_1': {'ridge': {'random_idx0': {'cate_true': array([ 0.01964673,  0.67548137,  0.01256466, ..., -0.08345062,
             0.20826006, -0.22013693]),
     'cate_pred': array([0.322864  , 0.05715146, 0.07463621, ..., 0.14558426, 0.26351616,
            0.31013753]),
     'ate_true': 0.12915102003284332,
     'ate_pred': 0.1964462118366355,
     'cate_mse': 0.6957057935431328,
     'ate_bias': 0.06729519180379218,
     'base_model_eval': {'mu1': {'mae': 0.4038342438314648,
       'r2': 0.06901228503301671},
      'mu0': {'mae': 0.2732396438908264, 'r2': 0.05553168254251639},
      'tau0': {'mae': 0.2731728968341871, 'r2': 0.09351356163528213},
      'tau1': {'mae': 0.4007390288386666, 'r2': 0.0009202576097605553},
      'propensity': {'auc': 0.4871026212492847, 'f1': 0.5466595020598245}}},
    'random_idx1': {'cate_true': array([ 0.04373459, -0.01348651, -0.12558549, ...,  2.88443699,
            -0.03312028,  0.24602149]),
     'cate_pred': array([ 0.0618785 , -0.00

In [12]:
def summarize_experiment_results(results_dict):
    records = []

    for setup_name, setup_dict in results_dict.items():
        for scenario_key in setup_dict:
            row = {
                ("setup_name", ""): setup_name,
                ("scenario_key", ""): scenario_key
            }

            for base_model in setup_dict[scenario_key]:
                avg_result = setup_dict[scenario_key].get(base_model, {}).get("average", {})
                mean_mse = avg_result.get("mean_cate_mse", np.nan)
                std_mse = avg_result.get("std_cate_mse", np.nan)
                mean_ate_pred = avg_result.get("mean_ate_pred", np.nan)
                std_ate_pred = avg_result.get("std_ate_pred", np.nan)
                mean_ate_true = avg_result.get("mean_ate_true", np.nan)
                std_ate_true = avg_result.get("std_ate_true", np.nan)
                mean_ate_bias = avg_result.get("mean_ate_bias", np.nan)
                std_ate_bias = avg_result.get("std_ate_bias", np.nan)
                runtime = avg_result.get("runtime", np.nan)

                row[(base_model, "CATE_MSE")] = f"{mean_mse:.3f} ± {std_mse:.3f}" if not pd.isna(mean_mse) else np.nan
                row[(base_model, "ATE_pred")] = f"{mean_ate_pred:.3f} ± {std_ate_pred:.3f}" if not pd.isna(mean_ate_pred) else np.nan
                row[(base_model, "ATE_true")] = f"{mean_ate_true:.3f} ± {std_ate_true:.3f}" if not pd.isna(mean_ate_true) else np.nan
                row[(base_model, "ATE_bias")] = f"{mean_ate_bias:.3f} ± {std_ate_bias:.3f}" if not pd.isna(mean_ate_bias) else np.nan
                row[(base_model, "runtime [s]")] = round(runtime) if not pd.isna(runtime) else np.nan

            records.append(row)

    df = pd.DataFrame.from_records(records)
    df.columns = pd.MultiIndex.from_tuples(df.columns)
    return df


In [13]:
print("T-Learner Results")
summary_df = summarize_experiment_results(results_dict)
summary_df

T-Learner Results


setup_name scenario_key          ridge                                \
                                CATE_MSE       ATE_pred       ATE_true   
0    RCT_0_5   scenario_1  0.688 ± 0.068  0.099 ± 0.047  0.126 ± 0.011   

                               
         ATE_bias runtime [s]  
0  -0.027 ± 0.050           1